<a href="https://colab.research.google.com/github/lexuanthinh/thinhlx-sparktut/blob/master/cs246-wordcount.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS246 - Colab 1
## Wordcount in Spark

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 217.8MB 59kB/s 
     |████████████████████████████████| 204kB 42.9MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=f011e63ae75aee160e2aa6034250733ac848363fbd452f0304bd3fe38bd07403
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark
openjdk-8-jdk-headless is already the newest version (8u242-b08-0ubuntu3~18.04).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
id='1SE6k_0YukzGd5wK-E4i6mG83nydlfvSa'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('pg100.txt')

If you executed the cells above, you should be able to see the file *pg100.txt* under the "Files" tab on the left panel.

### Your task

If you run successfully the setup stage, you are ready to work on the *pg100.txt* file which contains a copy of the complete works of Shakespeare.

Write a Spark application which outputs the number of words that start with each letter. This means that for every letter we want to count the total number of (non-unique) words that start with a specific letter. In your implementation **ignore the letter case**, i.e., consider all words as lower case. Also, you can ignore all the words **starting** with a non-alphabetic character.

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import pandas as pd
import pyspark


In [5]:
# YOUR
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

spark

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"



--2020-05-03 06:33:11--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.3.157.51, 34.197.27.35, 52.86.206.127, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.3.157.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  18.8MB/s    in 0.7s    

2020-05-03 06:33:12 (18.8 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
https://6d4082a7.ngrok.io


In [0]:
# CODE
    lines = spark.read.text("pg100.txt").rdd.map(lambda r: r[0])

In [10]:
lines.take(10)

['The Project Gutenberg EBook of The Complete Works of William Shakespeare, by',
 'William Shakespeare',
 '',
 'This eBook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever.  You may copy it, give it away or',
 're-use it under the terms of the Project Gutenberg License included',
 'with this eBook or online at www.gutenberg.org',
 '',
 '** This is a COPYRIGHTED Project Gutenberg eBook, Details Below **',
 '**     Please follow the copyright guidelines in this file.     **']

In [0]:
    from operator import add
    counts = lines.flatMap(lambda x: x.split(' ')) \
                  .map(lambda x: (x, 1)) \
                  .reduceByKey(add)

In [15]:
counts.take(10)

[('The', 3977),
 ('Project', 85),
 ('EBook', 2),
 ('of', 15649),
 ('Shakespeare', 45),
 ('', 506610),
 ('is', 7874),
 ('use', 266),
 ('anyone', 4),
 ('anywhere', 4)]

In [0]:
output = counts.collect()
for (word, count) in output:
  print("%s: %i" % (word, count))

In [21]:
words_count_with_schema = counts.map(lambda r: Row(word=r[0], freq=r[1]))
words_count = spark.createDataFrame(words_count_with_schema)
words_count.show()

+------+------------+
|  freq|        word|
+------+------------+
|  3977|         The|
|    85|     Project|
|     2|       EBook|
| 15649|          of|
|    45| Shakespeare|
|506610|            |
|  7874|          is|
|   266|         use|
|     4|      anyone|
|     4|    anywhere|
|  2227|          at|
|  2439|          no|
|     2|restrictions|
|     3| whatsoever.|
|  1341|         may|
|   529|         it,|
|   964|        give|
|   294|        away|
|     2|      re-use|
|  4809|        this|
+------+------------+
only showing top 20 rows



Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**!

In [24]:
most_freq = words_count.sort(desc('freq'))
most_freq.take(10)

[Row(freq=506610, word=''),
 Row(freq=23407, word='the'),
 Row(freq=19540, word='I'),
 Row(freq=18358, word='and'),
 Row(freq=15682, word='to'),
 Row(freq=15649, word='of'),
 Row(freq=12586, word='a'),
 Row(freq=10825, word='my'),
 Row(freq=9633, word='in'),
 Row(freq=9129, word='you')]